In [1]:
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from copy import deepcopy
import logging
FORMAT = '%(asctime)s, %(module)s, %(funcName)s, %(levelname)s:%(message)s'
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
logging.basicConfig(
    format=FORMAT,
    level=logging.INFO,#logging.DEBUG
    datefmt='%Y/%m, %H:%M',
    #just write to file
    #filename='urlMapCrawled.txt',#filemode='w'
    #write to file and print out, w: write/create, +: update, a: append
    handlers=[logging.FileHandler("urlMapCrawled.txt", mode='w+'), stream_handler]
)
class urlMapCrawler:
    #initiantion
    def __init__(self, urls=[]):
        self.primary_url=deepcopy(urls[0]) #exact copy
        self.crawed_urls = []
        self.urls_to_crawl = urls
    #find links in response, 
    #note that
        # 1) default: urljoin will concatenate url of website with href with ignorance of paths provided by url
        # 2) by defalut the code ignores url's paths with */*.js or ... 
        # 3) buttons with onclick events may not be consifered.
    def find_links(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path
    #if new related urls get discovered, they'll be added to sheduled lists to be crawled.
    def url_to_crawl(self, url):
        if url not in self.crawed_urls:
            self.urls_to_crawl.append(url)
    #main crawl function, check & add every urls in responses to crawling list.
    def crawl(self, url):
        html = requests.get(url).text
        for every_urls in self.find_links(url, html):
            #find the urls with deeper paths/subdirectories with respect to the provided url by user
            #check the None state of founded urls and be sure not included in crawling shedule.
            if every_urls != None and every_urls.startswith(self.primary_url):
                self.url_to_crawl(every_urls)
    #run function till we have url to crawl; in respect to crawl funrtion terms, available urls are subdirectories
    def run(self):
        while self.urls_to_crawl:
            #remove the crawel url using "pop" from the list
            first_url = self.urls_to_crawl.pop(0)
            if first_url not in self.crawed_urls:     
                logging.info(f' Crawled: {first_url}')
                try:
                    self.crawl(first_url)
                except Exception:
                    logging.exception(f' Crawling failed : {first_url}')
                finally:
                    self.crawed_urls.append(first_url)
#url list to be crawled
urls=['https://bakhshiali.github.io/publication',
     ]
if __name__ == '__main__':
    #call the run function in main to crawl url list.
    [urlMapCrawler(urls=[i]).run() for i in urls]
    #print the logged file
    with open("urlMapCrawled.txt") as f:
        print (f.read())
    #logging.shutdown()


2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publications/
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/8
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/7
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/6
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/5
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/4
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/3
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/2
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/1


2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publications/
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/8
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/7
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/6
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/5
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/4
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/3
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/2
2023/01, 14:35, 2401464476, run, INFO: Crawled: https://bakhshiali.github.io/publication/1

